In [11]:
import os
import numpy as np
from PIL import Image

# Параметры
TILE_SIZE = 150          # размер одного "тайла" (в пикселях)
GRID_SIZE = (100, 100)  # размер коллажа в тайлах (ширина, высота)

# Пути (Google Drive / Colab)
BASE_DIR = "/content/drive/MyDrive/useful/JPG_to_txt"
REFERENCE_PATH = os.path.join(BASE_DIR, "reference.jpg")
TILES_FOLDER = os.path.join(BASE_DIR, "tiles")
OUTPUT_PATH = os.path.join(BASE_DIR, "photomosaic.jpg")

# 1. Загрузка и обработка тайлов
def load_tiles(folder, tile_size):
    tiles = []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            try:
                img = Image.open(img_path).convert('RGB')
                img = img.resize((tile_size, tile_size), Image.LANCZOS)
                avg_color = np.array(img).mean(axis=(0, 1))  # средний RGB
                tiles.append((img, avg_color))
            except Exception as e:
                print(f"Ошибка при загрузке {img_path}: {e}")
    print(f"Загружено {len(tiles)} тайлов.")
    return tiles

# 2. Создание фотомозаики
def create_photomosaic(reference_path, tiles, grid_size, tile_size):
    ref = Image.open(reference_path).convert('RGB')
    ref = ref.resize(grid_size, Image.LANCZOS)  # уменьшаем до размера сетки (в тайлах)
    ref_array = np.array(ref)

    # Создаём итоговое изображение
    mosaic = Image.new('RGB', (grid_size[0] * tile_size, grid_size[1] * tile_size))

    # Проходим по каждому блоку
    for y in range(grid_size[1]):
        for x in range(grid_size[0]):
            target_color = ref_array[y, x]
            # Находим тайл с ближайшим средним цветом
            best_tile = min(tiles, key=lambda t: np.linalg.norm(t[1] - target_color))
            mosaic.paste(best_tile[0], (x * tile_size, y * tile_size))

    return mosaic

# Запуск
if __name__ == "__main__":
    tiles = load_tiles(TILES_FOLDER, TILE_SIZE)
    if not tiles:
        print("Нет тайлов! Проверьте папку:", TILES_FOLDER)
    else:
        mosaic = create_photomosaic(REFERENCE_PATH, tiles, GRID_SIZE, TILE_SIZE)
        mosaic.save(OUTPUT_PATH)
        print(f"✅ Фотомозаика сохранена: {OUTPUT_PATH}")

Загружено 148 тайлов.
✅ Фотомозаика сохранена: /content/drive/MyDrive/useful/JPG_to_txt/photomosaic.jpg


In [ ]:
#скрипт для создания html
import os

# Пути (те же, что и в основном скрипте)
BASE_DIR = "/content/drive/MyDrive/useful/JPG_to_txt"
IMAGE_PATH = os.path.join(BASE_DIR, "photomosaic.jpg")
HTML_OUTPUT_PATH = os.path.join(BASE_DIR, "photomosaic_viewer.html")

# Генерация HTML
html_content = f"""
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Фотомозаика</title>
    <style>
        body {{
            margin: 0;
            padding: 20px;
            background-color: #f0f0f0;
            font-family: Arial, sans-serif;
            text-align: center;
        }}
        h1 {{
            color: #333;
        }}
        .image-container {{
            overflow: auto;
            max-width: 100vw;
            max-height: 85vh;
            border: 1px solid #ccc;
            background: #fff;
            display: inline-block;
            margin-top: 10px;
        }}
        img {{
            display: block;
            /* Не масштабировать автоматически — показывать в оригинальном размере */
            width: auto;
            height: auto;
            max-width: none;
            max-height: none;
        }}
    </style>
</head>
<body>
    <h1>📸 Фотомозаика</h1>
    <div class="image-container">
        <img src="{os.path.basename(IMAGE_PATH)}" alt="Фотомозаика">
    </div>
    <p>Изображение сохранено в: <code>{IMAGE_PATH}</code></p>
</body>
</html>
"""

# Сохранение HTML-файла
with open(HTML_OUTPUT_PATH, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"✅ HTML-страница создана: {HTML_OUTPUT_PATH}")
print("💡 Чтобы открыть в Colab: загрузите файл photomosaic_viewer.html и откройте в браузере.")

✅ HTML-страница создана: /content/drive/MyDrive/useful/JPG_to_txt/photomosaic_viewer.html
💡 Чтобы открыть в Colab: загрузите файл photomosaic_viewer.html и откройте в браузере.
